## 1️⃣ Install Dependencies

In [ ]:
# Install Bark TTS, Supabase client, and optimization libraries
!pip install -q transformers scipy torch supabase optimum

print("✅ Dependencies installed!")

## 2️⃣ Upload SQL Files

Upload file berikut dari folder `database/`:
- `insert_listening_questions.sql`
- `insert_listening_additional.sql`

Klik tombol 📁 di sidebar kiri untuk upload files.

In [ ]:
import os
from pathlib import Path

# Check uploaded files
files_needed = ['insert_listening_questions.sql', 'insert_listening_additional.sql']
missing = [f for f in files_needed if not os.path.exists(f'/content/{f}')]

if missing:
    print(f"❌ Missing files: {missing}")
    print("Please upload them using the file browser on the left.")
else:
    print("✅ All SQL files uploaded!")

## 3️⃣ Configure Supabase

Get your credentials from: https://supabase.com/dashboard/project/_/settings/api

In [ ]:
# ⚠️ REPLACE WITH YOUR CREDENTIALS
SUPABASE_URL = "https://your-project.supabase.co"  # Replace this
SUPABASE_KEY = "your-anon-key-here"  # Replace this
BUCKET_NAME = "ptesol-audio"

# Validate
if "your-project" in SUPABASE_URL or "your-anon-key" in SUPABASE_KEY:
    print("❌ Please update SUPABASE_URL and SUPABASE_KEY with your actual credentials!")
else:
    print("✅ Supabase configured!")
    print(f"URL: {SUPABASE_URL}")
    print(f"Bucket: {BUCKET_NAME}")

## 4️⃣ Load Bark Model (GPU)

This will download ~5GB of model files. Takes 3-5 minutes.

In [ ]:
import torch
from transformers import AutoProcessor, BarkModel
import scipy
import numpy as np
import warnings
warnings.filterwarnings('ignore')

print("Loading Bark model...")
print(f"GPU Available: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"GPU: {torch.cuda.get_device_name(0)}")

device = "cuda" if torch.cuda.is_available() else "cpu"
processor = AutoProcessor.from_pretrained("suno/bark")
model = BarkModel.from_pretrained("suno/bark").to(device)

# Try to optimize with Better Transformer (requires optimum package)
try:

    model = model.to_bettertransformer()print("\n✅ Model loaded on", device.upper())

    print("✅ Better Transformer enabled (faster inference)")

except Exception as e:    print("Continuing with standard model (will be slower)")
    print(f"⚠️  Better Transformer not available: {e}")

## 5️⃣ Helper Functions

In [ ]:
import re
import random

def parse_sql_questions(sql_file):
    """Parse SQL INSERT statements to extract questions"""
    with open(sql_file, 'r', encoding='utf-8') as f:
        content = f.read()
    
    pattern = r"\((\d+),\s*'([^']+)',\s*'([^']+)',\s*'([^']+)',\s*'([^']+)',\s*'([^']+)',\s*'([^']+)',\s*'([^']+)',\s*'([^']+)',\s*([^,]+),\s*([^,]+),\s*NULL,\s*NULL\)"
    matches = re.findall(pattern, content)
    
    questions = []
    for match in matches:
        questions.append({
            'question_number': int(match[0]),
            'question_text': match[1],
            'section': match[2],
            'part': match[3],
            'option_a': match[4],
            'option_b': match[5],
            'option_c': match[6],
            'option_d': match[7],
            'correct_answer': match[8]
        })
    
    return sorted(questions, key=lambda x: x['question_number'])


def generate_part_a_conversation():
    """Generate casual 2-3 exchange conversation (Part A)"""
    templates = [
        [
            "Hey, did you finish the assignment yet?",
            "Almost done, just need to check my references.",
            "Great! Want to grab coffee after?"
        ],
        [
            "Are you going to the study group tonight?",
            "I'm not sure, I have to work on my presentation first."
        ],
        [
            "The library closes at 10, right?",
            "Actually, it closes at 11 on weekdays.",
            "Oh perfect, that gives us more time!"
        ],
        [
            "Did you understand the lecture today?",
            "Not really, the professor went too fast.",
            "Same here. Should we review together?"
        ]
    ]
    return random.choice(templates)


def generate_part_b_conversation():
    """Generate extended student-professor dialogue (Part B)"""
    templates = [
        [
            "Professor, do you have a moment to discuss my research proposal?",
            "Of course! Have a seat. What's your topic?",
            "I'm interested in studying renewable energy efficiency in urban areas.",
            "That's a fascinating area. Have you narrowed down your specific focus?",
            "I'm thinking about solar panel implementation in high-rise buildings.",
            "Excellent. You'll need to review the recent studies on building-integrated photovoltaics.",
            "I've started, but I'm having trouble finding data on cost-effectiveness.",
            "Check the Journal of Sustainable Energy. They published several papers on that last year.",
            "Thank you! Should I also look at case studies from other countries?",
            "Definitely. Singapore and Germany have excellent examples."
        ],
        [
            "Excuse me, Professor. I'm confused about the midterm exam format.",
            "Sure, what would you like to know?",
            "Will it be multiple choice or essay questions?",
            "It'll be a combination. Part one is multiple choice, part two is short essays.",
            "How should I prepare for the essay section?",
            "Focus on the main concepts we've covered. Be able to explain and apply them.",
            "Are there any specific topics I should prioritize?",
            "Review chapters five through eight. Those will be heavily featured.",
            "What about the group project? Does it affect the exam?",
            "The exam is separate, but understanding your project will help with the concepts."
        ]
    ]
    return random.choice(templates)


def generate_part_c_lecture():
    """Generate academic lecture monologue (Part C)"""
    templates = [
        [
            "Good morning, class. Today we'll be discussing the impact of climate change on coastal ecosystems. ",
            "As global temperatures rise, we're seeing significant changes in marine biodiversity. ",
            "One of the most affected areas is coral reefs. These ecosystems are incredibly sensitive to temperature changes. ",
            "Even a one-degree increase in water temperature can cause coral bleaching. ",
            "When corals bleach, they expel the algae living in their tissues, which they depend on for nutrition. ",
            "This not only affects the corals themselves but also the entire food chain that depends on them. ",
            "Fish populations decline, which impacts local fishing communities and economies. ",
            "Scientists are now studying coral resilience and looking for species that can adapt to warmer waters. ",
            "Some researchers are even transplanting heat-resistant corals to vulnerable reefs. ",
            "This field of study, called coral restoration, shows promise but requires significant resources and time."
        ],
        [
            "Let's talk about artificial intelligence and its role in modern education. ",
            "AI technology has evolved dramatically over the past decade. ",
            "We now have systems that can personalize learning experiences for individual students. ",
            "These systems analyze how a student learns and adapt the content accordingly. ",
            "For example, if a student struggles with mathematics, the AI can provide additional practice problems. ",
            "It can also identify patterns in learning behavior that teachers might miss. ",
            "However, there are important ethical considerations we must address. ",
            "Privacy concerns are paramount when collecting student data. ",
            "We must also ensure that AI doesn't replace human interaction, which is crucial for social development. ",
            "The goal is to use AI as a tool to enhance education, not to replace traditional teaching methods."
        ]
    ]
    return random.choice(templates)


def generate_audio(text, output_path, voice_preset="v2/en_speaker_3"):
    """Generate audio using Bark model"""
    if isinstance(text, list):
        text = " ".join(text)
    
    inputs = processor(text, voice_preset=voice_preset, return_tensors="pt").to(device)
    
    with torch.no_grad():
        audio_array = model.generate(**inputs, pad_token_id=10000)
    
    audio_array = audio_array.cpu().numpy().squeeze()
    sample_rate = model.generation_config.sample_rate
    
    scipy.io.wavfile.write(output_path, rate=sample_rate, data=audio_array)


print("✅ Helper functions loaded!")

## 6️⃣ Test Generation (1 sample)

Generate 1 audio file untuk test. Play audionya untuk pastikan hasilnya bagus.

In [ ]:
from IPython.display import Audio, display

# Create output directory
os.makedirs('/content/generated_audio', exist_ok=True)

# Generate test audio
test_text = ["Hey, how's your project going?", "Pretty good! I just finished the first draft.", "Nice! Want me to review it?"]
test_output = '/content/generated_audio/test_audio.wav'

print("Generating test audio...")
generate_audio(test_text, test_output)
print("✅ Test audio generated!\n")

# Play audio
display(Audio(test_output))
print("👆 Listen to the audio above. If it sounds good, proceed to full generation.")

## 7️⃣ Generate ALL Audio Files

⚠️ **This will take 30-60 minutes**

Progress bar akan muncul. Bisa minimize browser, tapi jangan close tab.

In [ ]:
from tqdm import tqdm
import time

# Parse SQL files
print("Parsing SQL files...")
questions = parse_sql_questions('/content/insert_listening_questions.sql')
additional = parse_sql_questions('/content/insert_listening_additional.sql')
all_questions = questions + additional

print(f"Found {len(all_questions)} questions\n")

# Generate audio files
generated_files = []
failed = []

for q in tqdm(all_questions, desc="Generating audio"):
    q_num = q['question_number']
    part = q['part'].lower().replace(' ', '')
    
    # Generate conversation based on part
    if 'part a' in part.lower():
        conversation = generate_part_a_conversation()
    elif 'part b' in part.lower():
        conversation = generate_part_b_conversation()
    else:  # Part C
        conversation = generate_part_c_lecture()
    
    # Generate filename
    filename = f"listening_{q_num:03d}_{part}.wav"
    output_path = f"/content/generated_audio/{filename}"
    
    try:
        generate_audio(conversation, output_path)
        generated_files.append(filename)
    except Exception as e:
        print(f"\n❌ Failed: {filename} - {e}")
        failed.append((filename, str(e)))
    
    # Prevent overheating
    time.sleep(0.5)

# Summary
print(f"\n✅ Generated: {len(generated_files)}/{len(all_questions)}")
if failed:
    print(f"❌ Failed: {len(failed)}")
    for f, e in failed:
        print(f"  - {f}: {e}")

## 8️⃣ Upload to Supabase Storage

Upload semua audio files ke Supabase dan generate SQL UPDATE statements.

In [ ]:
from supabase import create_client, Client
from pathlib import Path

# Initialize Supabase client
supabase: Client = create_client(SUPABASE_URL, SUPABASE_KEY)

print("Uploading to Supabase Storage...\n")

sql_updates = []
uploaded_count = 0
upload_failed = []

audio_dir = Path('/content/generated_audio')
wav_files = [f for f in audio_dir.glob('*.wav') if f.name != 'test_audio.wav']

for wav_file in tqdm(wav_files, desc="Uploading"):
    filename = wav_file.name
    
    # Extract question number from filename (listening_001_parta.wav -> 1)
    match = re.search(r'listening_(\d+)_', filename)
    if not match:
        print(f"⚠️  Skipping {filename}: Can't extract question number")
        continue
    
    question_number = int(match.group(1))
    
    try:
        # Read file
        with open(wav_file, 'rb') as f:
            file_data = f.read()
        
        # Upload to Supabase Storage
        supabase.storage.from_(BUCKET_NAME).upload(
            path=filename,
            file=file_data,
            file_options={"content-type": "audio/wav", "upsert": "true"}
        )
        
        # Get public URL
        public_url = supabase.storage.from_(BUCKET_NAME).get_public_url(filename)
        
        # Generate SQL UPDATE
        sql_updates.append(
            f"UPDATE questions SET audio_url = '{public_url}' WHERE question_number = {question_number};"
        )
        
        uploaded_count += 1
        
    except Exception as e:
        print(f"\n❌ Upload failed: {filename} - {e}")
        upload_failed.append((filename, str(e)))

# Save SQL updates to file
sql_file = '/content/audio_urls_update.sql'
with open(sql_file, 'w', encoding='utf-8') as f:
    f.write("-- Auto-generated SQL UPDATE statements\n")
    f.write("-- Run this in Supabase SQL Editor to update audio URLs\n\n")
    f.write("\n".join(sql_updates))

print(f"\n✅ Uploaded: {uploaded_count}/{len(wav_files)} files")
print(f"✅ SQL file saved: {sql_file}")

if upload_failed:
    print(f"\n❌ Upload failures: {len(upload_failed)}")
    for f, e in upload_failed:
        print(f"  - {f}: {e}")

## 9️⃣ Download SQL File

Download `audio_urls_update.sql` dan jalankan di Supabase SQL Editor.

In [ ]:
from google.colab import files

# Download SQL file
files.download('/content/audio_urls_update.sql')

print("\n✅ SQL file downloaded!")
print("\nNext steps:")
print("1. Go to Supabase Dashboard > SQL Editor")
print("2. Open audio_urls_update.sql")
print("3. Copy-paste the SQL and run it")
print("4. Done! Audio URLs are now in your database")

## 🎉 Complete!

### Summary:
- ✅ Generated 200 audio files with Bark TTS
- ✅ Uploaded to Supabase Storage
- ✅ Created SQL UPDATE statements

### Next Steps:
1. Run the SQL file in Supabase
2. Test audio playback in your Next.js app
3. Deploy to Vercel

---

**💡 Tips:**
- Jika ada audio yang kurang bagus, generate ulang dengan menjalankan cell 6 (test generation) dengan text yang berbeda
- Audio files sudah ada di Supabase, jadi tidak perlu upload ulang kalau konten sama
- Untuk re-generate specific audio, cukup upload ulang dengan nama file yang sama (upsert: true)